In [ ]:
from collections import Counter
import heapq
from graphviz import Digraph
import os

class HuffmanNode:
    def __init__(self, char: str = '', freq: int = 0):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

class HuffmanVisualizer:
    def __init__(self, text: str):
        self.text = text
        self.freq_dict = Counter(text)
        self.root = self.build_huffman_tree()
        self.codes = {}
        
    def build_huffman_tree(self) -> HuffmanNode:
        # Create initial heap from frequencies
        heap = []
        for char, freq in self.freq_dict.items():
            heapq.heappush(heap, HuffmanNode(char, freq))
        
        # Build tree by combining nodes
        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            internal = HuffmanNode('', left.freq + right.freq)
            internal.left = left
            internal.right = right
            heapq.heappush(heap, internal)
        
        return heap[0]
    
    def _generate_codes(self, node: HuffmanNode, code: str = ''):
        if node.char:  # Leaf node
            self.codes[node.char] = code
            return
        
        if node.left:
            self._generate_codes(node.left, code + '0')
        if node.right:
            self._generate_codes(node.right, code + '1')
    
    def visualize_tree(self, filename: str = 'huffman_tree'):
        dot = Digraph(comment='Huffman Tree')
        dot.attr(rankdir='TB')  # Top to bottom layout
        
        # Node attributes
        dot.attr('node', shape='circle', style='filled')
        
        def add_nodes(node: HuffmanNode, node_id: str = 'root'):
            if node.char:
                # Leaf node
                label = f'{node.char}\n{node.freq}'
                if node.char == ' ':
                    label = f'space\n{node.freq}'
                dot.node(node_id, label, fillcolor='lightblue')
            else:
                # Internal node
                dot.node(node_id, str(node.freq), fillcolor='lightgray')
            
            # Add children
            if node.left:
                left_id = f'{node_id}_l'
                dot.edge(node_id, left_id, '0')  # Label edge with '0' for left
                add_nodes(node.left, left_id)
            
            if node.right:
                right_id = f'{node_id}_r'
                dot.edge(node_id, right_id, '1')  # Label edge with '1' for right
                add_nodes(node.right, right_id)
        
        # Build the tree visualization
        add_nodes(self.root)
        
        # Generate codes for printing
        self._generate_codes(self.root)
        
        # Create a subgraph for compression info
        with dot.subgraph(name='cluster_0') as c:
            c.attr(label='Compression Information')
            c.attr('node', shape='box', style='filled', fillcolor='white')
            
            # Calculate compression stats
            original_bits = len(self.text) * 8
            compressed_bits = sum(len(self.codes[char]) * freq 
                                for char, freq in self.freq_dict.items())
            compression_ratio = ((original_bits - compressed_bits) / original_bits) * 100
            
            # Add compression statistics
            stats = (f'Original Size: {original_bits} bits\n'
                    f'Compressed Size: {compressed_bits} bits\n'
                    f'Compression Ratio: {compression_ratio:.2f}%')
            c.node('stats', stats)
            
            # Add Huffman codes
            codes_text = 'Huffman Codes:\n' + '\n'.join(
                f"'{k if k != ' ' else 'space'}': {v}"
                for k, v in sorted(self.codes.items())
            )
            c.node('codes', codes_text)
        
        # Save visualization
        dot.render(filename, view=True, format='png', cleanup=True)

def visualize_huffman_coding(text: str, filename: str = 'huffman_tree'):
    """Create and display Huffman tree visualization for given text"""
    visualizer = HuffmanVisualizer(text)
    visualizer.visualize_tree(filename)
    print(f"Visualization saved as {filename}.png")

# Example usage
if __name__ == "__main__":
    sample_text = "zbbu buu zzbbeeebezb"
    visualize_huffman_coding(sample_text)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

: 